In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [2]:
bandit_arms = [0.2, 0, -0.2, -2]
num_arms = len(bandit_arms)
def pull_bandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        # positive reward
        return 1
    else:
        # negative reward
        return -1

In [3]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_arms]))
outputs = tf.nn.softmax(weights)

reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

responsible_output = tf.slice(outputs, action_holder, [1])
loss = -(tf.log(responsible_output) * reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

In [ ]:
total_episodes = 1000
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        # choose action according to boltzmann distribution
        actions = sess.run(outputs)
        a = np.random.choice(actions, p=actions)
        action = np.argmax(actions == a)
        reward = pull_bandit(bandit_arms[action])
        _, resp, ww = sess.run([update, responsible_output, weights], feed_dict={reward_holder:[reward], action_holder:[action]})
        
        total_reward[action] += reward
        if i % 50 == 0:
            print('Running reward for the ' + str(num_arms) + ' arms of the bandi: ' + str(total_reward))
        i += 1

print('The agent thinks arm ' + str(np.argmax(ww) + 1) + ' is the most promising...')
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print('...and it was right!')
else:
    print('...but it was wrong!')